## 深度学习大挑战

案例说明：手写体识别，使用卷积神经网络（CNN）

手写体识别，是人工智能中的经典难题。这个案例中，我们需要搭建神经网络，让计算机识别大量的手写体的图片，然后手写一张图片让计算机来识别。

用计算机处理手写体文字在之前一直是个难题，因为每个人的书写风格、书写习惯都不同，很难通过传统算法对书写内容进行判断。而卷积神经网络(Convolutional Neural Networks，简称 CNN)在处理这类问题上有非常好的表现，通过准备好的大量手写体训练样本，可以让计算机“学会”认识手写文字。

案例选择了keras框架，需要先安装keras和tensorflow。

# 图象处理

命名格式为种类+序号

这里我们的种类1代表的是手写数字壹，之后进行训练时，种类数字代表的就是图片的标签。

批量修改图片分辨率并重命名的代码(CV2版本)

## 注意，路径或文件名中不能包含中文

In [4]:
import os
import cv2
fileName = '1'
path = 'photo1'
s = os.listdir(path)
count=1
for i in s:
    #如果是图片文件，则裁剪为100*100并重命名
    if i.endswith('.jpg') or i.endswith('.png') or i.endswith('.jpeg'):
        document = os.path.join(path,i)
        img = cv2.imread(document)
        img = cv2.resize(img,(100,100))
        cv2.imwrite(path+os.sep+fileName+str(count)+'.jpg',img)
        count=count+1

## 数据处理

In [ ]:
import numpy as np
from keras import utils
from PIL import Image

ima_train = os.listdir('./train')
ima_test = os.listdir('./test')
# 图片其实就是一个矩阵（每一个像素都是0-255之间的数）（100*100*1）

train_image = []
test_image = []
#将图片转化为灰度图矩阵后,放在列表里

for i in ima_train:
    img=Image.open('./train/' + i ).convert('L')
    img = np.expand_dims(np.array(img),axis=-1)
    train_image.append(img)

for i in ima_test:
    img=Image.open('./train/' + i ).convert('L')
    img = np.expand_dims(np.array(img),axis=-1)
    test_image.append(img)

    
train_image = np.array(train_image)
test_image = np.array(test_image)
# #提取首字符作为标签，储存到列表train_label中
train_label = []
test_lable = []
for filename in ima_train:
    train_label.append(int(filename[0]))
for filename in ima_test:
    test_lable.append(int(filename[0]))
    
# 标签（0/1/2/3）(840,)
train_label = np.array(train_label)
test_lable = np.array(test_lable)

# 4.把标签转换为特殊矩阵
train_label = utils.to_categorical(train_label)
test_lable = utils.to_categorical(test_lable)

train_image = train_image.astype('float32')
test_image = test_image.astype('float32')

train_image /= 255
test_image /= 255

print(train_image.shape)  # (数据量, 100, 100, 1)
print(train_label.shape)  # (数据量, 4)


## 搭建模型

In [ ]:
#### import keras
from keras import layers
model=keras.models.Sequential()
#第一层要设置输入图片的尺寸，即100*100像素、1个颜色通道
model.add(layers.Conv2D(64,(3,3),activation='relu',input_shape=(100,100,1)))  
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPool2D())
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPool2D())
#在全连接之前，需要将二维图片数据转换成一维数组
model.add(layers.Flatten()) 
#实现卷积
model.add(layers.Dense(256,activation='relu'))
#为了防止过拟合，Dropout层会随机的丢弃一部分神经网络连接
model.add(layers.Dropout(0.5))  
#使用softmax处理多分类问题
model.add(layers.Dense(4,activation='softmax')) 

## 编译、训练模型

In [ ]:
from keras.optimizers import SGD
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])#
# epochs一共进行的轮数，batch_size每次训练数据量
model.fit(train_image, train_label, batch_size=10, epochs=32)

In [ ]:
# 保存模型文件
model.save('./nums.h5', overwrite=True)
# 评估模型
score = model.evaluate(test_image, test_lable, batch_size=10)
print(score)

## 进行预测获得结果（直接调用模型）

In [ ]:
from PIL import Image
import numpy as np
from keras.models import load_model  
# 1.上传图片
img_path = ('test.jpg')
# 2.预处理图片
img = Image.open(img_path)
img = img.resize((100, 100), Image.BILINEAR)
img = img.convert('L')
img = np.expand_dims(np.array(img),axis=-1)
test_image = []
test_image.append(img)
test_image = np.array(test_image)
test_image = (test_image.astype('float32'))/255
# 3.加载模型
model = load_model('nums.h5')
# 4.预测类别，返回预测的序号
classes = model.predict_classes(test_image)[0]
target = ['壹', '贰','叁','肆','伍','陆','柒','捌','玖','拾']
# 5.打印结果
print("识别结果为:" + target[classes])